# 3.3 线性回归的简洁实现

使用 pytorch 来实现线性回归

In [1]:
import torch
import torch.nn as nn
import numpy as np 

## 3.3.1 生成数据集

`features` 是训练特征，`labels` 是标签。

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)


## 3.3.2 读取数据

Pytorch提供了`data`包来读取数据。

In [3]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [7]:
# 打印小批量数据
for X, y in data_iter:
    print(X, y)
    break

tensor([[-0.1237, -0.7421],
        [-0.6384, -0.9239],
        [-0.9861, -2.4775],
        [-0.2122, -0.3304],
        [-2.0808,  1.8146],
        [ 0.8074,  0.3440],
        [-0.1771,  0.3673],
        [ 0.4034,  0.5216],
        [ 1.2224,  0.6964],
        [ 0.2710, -0.3964]]) tensor([ 6.4712,  6.0681, 10.6471,  4.9003, -6.1153,  4.6477,  2.5842,  3.2305,
         4.2634,  6.0934])


## 3.3.3 定义模型

导入 `torch.nn` 模块。`nn` 的核心数据机构是 `Module`，它是一个抽象概念，既可以表示神经网络中的一个层，也可以表示包含很多层的神经网络。

In [9]:
# 实现线性模型
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


还可以使用`nn.Sequential`来更加方便的搭建网络，`Sequential`是一个有序容器，网络层将按照在传入`Sequential`的顺序依次添加到计算图中。

In [11]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
)

print(net)

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)


In [14]:
# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module....
print(net)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [4]:
# 写法三
# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [15]:
# 查看模型所有的可学习参数
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.1395, -0.4576]], requires_grad=True)
Parameter containing:
tensor([0.1007], requires_grad=True)


## 3.3.4 初始化模型参数

在使用net前，我们需要初始化模型参数，如线性回归模型中的权重和偏差。我们通过`init.normal_`将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布。偏差会初始化为零。

In [5]:
from torch.nn import init 

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)


Parameter containing:
tensor([0.], requires_grad=True)

## 3.3.5 定义损失函数

In [6]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

`torch.optim` 提供了很多常用的优化算法比如SGD、Adam和RMSProp等。

In [7]:
import torch.optim as optim 

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [24]:
# 为不同的子网络设置不同的学习率
optimizer = optim.SGD([
    # 如果对某个参数不指定学习率，就是使用最外层的默认学习率
    {'params': net.subnet1.parameters()},
    {'params': net.subnet2.parameters(), 'lr': 0.01}
], lr=0.03)


In [ ]:
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

## 3.3.7 训练模型

In [8]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000327
epoch 2, loss: 0.000046
epoch 3, loss: 0.000106


In [9]:
# 比较模型参数与真实参数
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9997, -3.4000]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1997], requires_grad=True)


## 小结

- 使用PyTorch可以更简洁地实现模型
- `torch.utils.data`模块提供了有关数据处理的工具，`torch.nn`模块定义了大量神经网络的层，`torch.nn.init`模块定义了各种初始化方法，`torch.optim`模块提供了很多常用的优化算法